In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

In [ ]:
# path to the dataset directory
input_dir = "<path_to_dataset>"
files = os.listdir(input_dir)

In [ ]:
# Read folder names as labels and images underneath
i=0
last=[]
images=[]
labels=[]
temp = input_dir

# Read folders in the main dataset folder, one at a time
for files in input_dir:
  sub_folder_index = temp.index(files)
  label = sub_folder_index

  path = input_dir+'/'+sub_folder
  sub_folder_images= os.listdir(path)

  # Read images in the sub folder, one at a time
  for image in sub_folder_images:
    image_path = path+'/'+image
    print(image_path)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image= cv2.resize(image,(48,48))
    images.append(image)
    labels.append(label)
    i+=1
  last.append(i)

# Declare x and y
images_x = np.array(images)
labels_y = np.array(labels)
# We divide image pixels by 255 to reduce computation power
images_x = images_x/255

# encoding the labels
num_of_classes = 7
labels_y_encoded = tf.keras.utils.to_categorical(labels_y,num_classes=num_of_classes)

# Split into 75:25 train and test
X_train, X_test, Y_train, Y_test= train_test_split(images_x, labels_y_encoded,test_size=0.25, random_state=10)

In [ ]:
# CNN Architecture
# Define the dropout rate
dropout_rate = 0.1
# Define the L2 regularization rate
l2_rate = 0.001
# Input Layer Of 48x48 pixels
input = Input(shape = (48,48,1))
# Convolutional Layer with 32 filters of size 3x3, with L2 regularization
conv_1 = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(l2_rate))(input)
# Dropout Layer with 0.1 dropout rate
conv_1 = Dropout(dropout_rate)(conv_1)
# Activation Layer with ReLU activation function
conv_1 = Activation('relu')(conv_1)
# MaxPooling Layer with 2x2 pool size
pool_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)
# Convolutional Layer with 64 filters of size 3x3, with L2 regularization
conv_2 = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(l2_rate))(pool_1)
# Dropout Layer with 0.1 dropout rate
conv_2 = Dropout(dropout_rate)(conv_2)
# Activation Layer with ReLU activation function
conv_2 = Activation('relu')(conv_2)
# MaxPooling Layer with 2x2 pool size
pool_2 = MaxPooling2D(pool_size = (2,2)) (conv_2)
# Convolutional Layer with 128 filters of size 3x3, with L2 regularization
conv_3 = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(l2_rate))(pool_2)
# Dropout Layer with 0.1 dropout rate
conv_3 = Dropout(dropout_rate)(conv_3)
# Activation Layer with ReLU activation function
conv_3 = Activation('relu')(conv_3)
# MaxPooling Layer with 2x2 pool size
pool_3 = MaxPooling2D(pool_size = (2,2)) (conv_3)
# Convolutional Layer with 256 filters of size 3x3, with L2 regularization
conv_4 = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(l2_rate))(pool_3)
# Dropout Layer with 0.1 dropout rate
conv_4 = Dropout(dropout_rate)(pool_3)
# Activation Layer with ReLU activation function
conv_4 = Activation('relu')(conv_4)
# MaxPooling Layer with 2x2 pool size
pool_4 = MaxPooling2D(pool_size = (2,2)) (conv_4)
# Flatten Layer to convert 3D matrix to 1D vector
flatten = Flatten()(pool_4)
# Relu Layer with 128 neurons
dense_1 = Dense(128,activation='relu')(flatten)
# Dropout Layer with 0.2 dropout rate
drop_1 = Dropout(dropout_rate *2)(dense_1)
# Sigmoid Layer with 7 neurons for 7 classes
output = Dense(7,activation="sigmoid")(drop_1)

In [ ]:
# Create the model
model = Model(inputs=input,outputs=output)
# Adams optimizer with learning rate of 0.001
# Categorical crossentropy loss function
model.compile(optimizer="adam", loss=["categorical_crossentropy"], metrics=['accuracy'])
model.summary()

In [ ]:
# Configure Model Checkpoint
file_path='./output/emotion_model.keras'
# Save the model after every epoch
checkpointer = ModelCheckpoint(file_path, monitor='loss',verbose=1,save_best_only=True,
                               save_weights_only=False, mode='auto',save_freq='epoch')
# Set the callback list to include the checkpointer
callback_list=[checkpointer]

In [ ]:
# Train the model
# Fit the model with training data, validation data, and the callback list
save = model.fit(X_train,Y_train,batch_size=32,validation_data=(X_test,Y_test),epochs=50,callbacks=[callback_list])

In [ ]:
# Checking the train and test loss and accuracy values from the neural network above.
train_loss = save.history['loss']
test_loss = save.history['val_loss']
train_accuracy = save.history['accuracy']
test_accuracy = save.history['val_accuracy']

# Plotting a line chart to visualize the loss and accuracy values by epochs.
fig, ax = plt.subplots(ncols=2, figsize=(15,7))
ax = ax.ravel()
ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)
ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)
ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)
ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)
ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)
ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)
fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)

In [ ]:
# save the model to a file
model.save('./output/emotion_model.h5')